# 데이터 전처리
Colab의 Local disk의 여유 공간은 약 30~60GB 정도로 AI Hub에 있는 데이터를 풀어내기에는 적합하지 않다. Local에 있는 Disk를 이용하여 Resize를 수행하기로 한다.

## 1.다운로드 데이터 분석
AI-Hub에서 다운로드 받은 데이터는 아래와 같은 구조를 가진다. 


```bash
랜드마크 이미지
├─Training
│  └─세종특별자치시
│          [라벨]세종특별자치시.zip
│          [원천]세종특별자치시_001.zip
└─Validation
    └─세종특별자치시
            [라벨]세종특별자치시.zip
            [원천]세종특별자치시_001.zip
```
테스트와 검증 데이터가 분리되어 있으니 그대로 이용하고, [라벨] 파일은 metadata가 있는 json파일이다. 현 시점에서는 필요없으니 무시한다. 즉, [원천] 파일만 압축을 풀어 Resize를 수행하면 된다.


## 2.데이터 압축 해제
중간폴더의 지역명은 필요없으니 무시하고 디렉토리를 구성하여 압축을 해제한다.

In [3]:
import os, glob

base_dir = '/content/랜드마크 이미지/'
extract_dir = '/content/extracts'

In [ ]:

for dtype in os.listdir(base_dir):
    dst_dir = os.path.join(extract_dir, dtype)
    os.makedirs(dst_dir, exist_ok=True)
    for file_name in glob.glob(os.path.join(base_dir, dtype) + '/**/*원천*', recursive=True):
        file_name = '\'' + file_name + '\''
        !unzip -o -qq {file_name} -d {dst_dir}

## 3.RESIZE
압축 해제한 데이터의 구조는 아래와 같다. 파일 확장자명은 JPG or jpg로 되어 있으니 해당 파일을 찾아
OpenCV로 0.1배로 Resize하여 저장한다.

```bash
6.25격전지개미고개
  └─6.25격전지개미고개_001_40613647.JPG
    6.25격전지개미고개_003_40613648.JPG
    6.25격전지개미고개_004_40613649.JPG
    6.25격전지개미고개_005_40613650.JPG
    6.25격전지개미고개_006_40613651.JPG
    (...)
```

In [5]:
import cv2
from tqdm.notebook import tqdm

resized_dir = '/content/resizeds'

In [ ]:
for dtype in os.listdir(extract_dir):
    src_dir = os.path.join(extract_dir, dtype)
    dst_dir = os.path.join(resized_dir, dtype)
    for cls in os.listdir(src_dir):
        os.makedirs(os.path.join(dst_dir, cls), exist_ok=True)
    
    src_list = glob.glob('**/*.JPG', root_dir=os.path.join(extract_dir, dtype))
    print('Dataset Type: {}'.format(dtype))
    for fname in tqdm(src_list):
        img = cv2.imread(os.path.join(src_dir, fname))
        resized_img = cv2.resize(img, (0, 0), fx=0.1, fy=0.1, interpolation=cv2.INTER_AREA)
        cv2.imwrite(os.path.join(dst_dir, fname), resized_img)
    


## 압축
GPU사용을 위하 Colab에 올리기 위해 다시 zip 파일로 압축한다.


In [ ]:
!cd '/content'
!zip -r 'resize_0.1.zip' './resizeds'